<a href="https://colab.research.google.com/github/sudiptiwari/matrix-multiplication-optimization/blob/main/Matrix_Multiplication_Optimization_by_Strassen_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import math
import time

## Splitting the Matrix into 4 blocks

In [7]:
def split_matrix(matrix : np.matrix):
  """
  Splits the matrix into 4 parts
  ( A11 | A12 )
  -------------
  ( A21 | A22 )
  """
  row, column = matrix.shape
  mid_row = row // 2 # Integer Division to get the mid-point
  mid_column = column // 2 # Integer Division to get the mid-point
  return matrix[:mid_row, :mid_column], matrix[:mid_row, mid_column:], matrix[mid_row:, :mid_column], matrix[mid_row:, mid_column:]

In [8]:
A, B, C, D = split_matrix(np.matrix([
    [1,2,3,4],
    [3,4,5,6],
    [5,6,7,8],
    [8,9,10,11]
]))

print("A11: \n",A)
print("A12: \n",B)
print("A21: \n",C)
print("A22: \n",D)

A11: 
 [[1 2]
 [3 4]]
A12: 
 [[3 4]
 [5 6]]
A21: 
 [[5 6]
 [8 9]]
A22: 
 [[ 7  8]
 [10 11]]


## Implementation of Strassen's Algorithm

In [9]:
def multiplication_by_strassen_method(A : np.matrix, B: np.matrix):
  """
  Recursive Multiplication of two matrices A and B
  """
  if (A.shape == B.shape == (1,1)):
    return A * B

  # Splitting the matrix into Four Blocks (sub-matrices) : A11, A12, A21, A22
  A11, A12, A21, A22 = split_matrix(A)
  B11, B12, B21, B22 = split_matrix(B)

  # Perform 7 multiplications
  P1 = multiplication_by_strassen_method(A11 + A22, B11 + B22)  # P1 = (A11 + A22) * (B11 + B22)
  P2 = multiplication_by_strassen_method(A21 + A22, B11)        # P2 = (A21 + A22) * B11
  P3 = multiplication_by_strassen_method(A11, B12 - B22)        # P3 = A11 * (B12 - B22)
  P4 = multiplication_by_strassen_method(A22, B21 - B11)        # P4 = A22 * (B21 - B11)
  P5 = multiplication_by_strassen_method(A11 + A12, B22)        # P5 = (A11 + A12) * B22
  P6 = multiplication_by_strassen_method(A21 - A11, B11 + B12)  # P6 = (A21 - A11) * (B11 + B12)
  P7 = multiplication_by_strassen_method(A12 - A22, B21 + B22)  # P7 = (A12 - A22) * (B21 + B22)

  # Combine the 7 multiplications to get the answer(for each block)
  C11 = P1 + P4 - P5 + P7
  C12 = P3 + P5
  C21 = P2 + P4
  C22 = P1 + P3 - P2 + P6

  # Merge the blocks(sub-matrices) into single Matrix
  C_upper = np.hstack((C11, C12))
  C_lower = np.hstack((C21, C22))
  C_final_result = np.vstack((C_upper, C_lower))
  return C_final_result

## Perform padding to ensure 2^n dimension of two Square Matrix Inputs
- Ask Square matrix with same dimensions(any dimensions) as input.
- Convert the Square Matrices into the dimension of 2^k by padding.

In [16]:
def perform_padding_and_matrix_multiplication_by_strassen_method(A : np.matrix, B : np.matrix):
  """
  Pre-process the Input Matrices so that their dimension is in the form of 2^k by performing padding.
  Also ensure that both the Matrices have same dimension of 2^k.
  """

  # Ask/Ensure Square matrix with same dimensions as Input
  if( (A.shape[0] != A.shape[1]) or (B.shape[0] != B.shape[1]) or (A.shape != B.shape) ):
    print("Please enter square matrix with same dimensions.")
    return

  # Padding is done if the matrices are not in the form 2^k
  number_of_rows_initial = A.shape[0]
  next_power_of_2_greater_than_number_of_rows_initial = 2 ** math.ceil(math.log2(number_of_rows_initial))

  # Renaming for Simplicity in code
  n = number_of_rows_initial
  m = next_power_of_2_greater_than_number_of_rows_initial

  A_pad = np.pad(A, ((0, m - n), (0, m - n)), mode='constant', constant_values=0)
  B_pad = np.pad(B, ((0, m - n), (0, m - n)), mode='constant', constant_values=0)

  C_pad = multiplication_by_strassen_method(A_pad, B_pad)

  # Removing the padding before returning
  return C_pad[:n, :n]

In [22]:
# Sample 4*4 matrices
A = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])

B = np.array([[16, 15, 14, 13],
              [12, 11, 10, 9],
              [8, 7, 6, 5],
              [4, 3, 2, 1]])

# Perform Strassen matrix multiplication
start_time = time.time()
result = perform_padding_and_matrix_multiplication_by_strassen_method(A, B)
end_time = time.time()
strassen_time = end_time - start_time

print("Matrix A:\n", A)
print("Matrix B:\n", B)
print("Result of Strassen Multiplication:\n", result)
print(f"Time taken by Strassen's method: {strassen_time} seconds\n")

Matrix A:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
Matrix B:
 [[16 15 14 13]
 [12 11 10  9]
 [ 8  7  6  5]
 [ 4  3  2  1]]
Result of Strassen Multiplication:
 [[ 80  70  60  50]
 [240 214 188 162]
 [400 358 316 274]
 [560 502 444 386]]
Time taken by Strassen's method: 0.0013666152954101562 seconds



## Perform Matrix Multiplication by Naive Algorithm

In [26]:
def naive_matrix_multiplication(A : np.matrix, B : np.matrix):
    """
    Perform naive matrix multiplication of matrices A and B.
    """
    n = len(A)
    C = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]

    return C

In [28]:
start_time = time.time()
result_by_naive_method = naive_matrix_multiplication(A,B)
end_time = time.time()
naive_method_time = end_time - start_time

print("Matrix A:\n", A)
print("Matrix B:\n", B)
print("Result of In-built Multiplication:\n", result_by_naive_method)
print(f"Time taken by Naive Matrix Multiplication Method: {naive_method_time} seconds\n")

Matrix A:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
Matrix B:
 [[16 15 14 13]
 [12 11 10  9]
 [ 8  7  6  5]
 [ 4  3  2  1]]
Result of In-built Multiplication:
 [[ 80.  70.  60.  50.]
 [240. 214. 188. 162.]
 [400. 358. 316. 274.]
 [560. 502. 444. 386.]]
Time taken by Naive Matrix Multiplication Method: 0.00034809112548828125 seconds



## Perform Matrix Multiplication by Inbuilt NumPy method

In [23]:
start_time = time.time()
result_by_numpy_inbuilt_method = A@B
end_time = time.time()
numpy_time = end_time - start_time

print("Matrix A:\n", A)
print("Matrix B:\n", B)
print("Result of In-built Multiplication:\n", result_by_numpy_inbuilt_method)
print(f"Time taken by NumPy's in-built method: {numpy_time} seconds\n")

Matrix A:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
Matrix B:
 [[16 15 14 13]
 [12 11 10  9]
 [ 8  7  6  5]
 [ 4  3  2  1]]
Result of In-built Multiplication:
 [[ 80  70  60  50]
 [240 214 188 162]
 [400 358 316 274]
 [560 502 444 386]]
Time taken by NumPy's in-built method: 0.00015616416931152344 seconds

